In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import lxml
import requests
import re
import pandas as pd
import numpy as np
import csv
from lxml import html
import requests
from langdetect import detect
import math
import csv
import os

# Question 1

## URLs

In [ ]:
# Needs to be changed to 201 to 301
outfile = open("test.txt", "w")
for i in range(201,301):
    
    base_url = "https://www.goodreads.com/list/show/1.Best_Books_Ever?page="+str(i)

    html = urlopen(base_url)
    soup = BeautifulSoup(html.read(), features="lxml")

    links = []
    regex = "/book/show/."

    for link in soup.find_all("a"):
        if re.match(regex, str(link.get("href"))):
            links.append("https://www.goodreads.com"+link.get("href"))
    
    #issues of duplicates        
    links = list(dict.fromkeys(links))
    
    for link in links:
        outfile.write(link+"\n")
outfile.close()

## HTMLs


In [ ]:
# First, I make sure i am in the directory that has the text file with the URLS
path = "C:/Users/Guillaume/Desktop/ADM_HM3"
os.chdir(path)

i = 1
page = 1

with open("test_urls.txt", "r") as urls:
    # I previously created the 100 directories, from page 201 to page 300
    # I change the working directory for which i want the HTMLs 
    path = "C:/Users/Guillaume/Desktop/ADM_HM3/page_201"
    os.chdir(path)

    for url in urls.readlines()[0:]:

        html = urlopen(url)

        soup = BeautifulSoup(html.read(), 'html.parser')
        
        # The logig here is not the best but it works- 
        # Basically, once every 100 urls treated I chance the directory
        
        if (i%101) != 0:
                # We stay in the current directory
            with open("article_"+str(i)+".html", "w", encoding="utf-8") as outfile:
                outfile.write(str(soup))

            i = i + 1

        else :

            page = page + 1
            path = "C:/Users/Guillaume/Desktop/ADM_HM3/page_20"+str(page)
            os.chdir(path)

            with open("article_"+str(i)+".html", "w", encoding="utf-8") as outfile:
                outfile.write(str(soup))

## Parsingin HTMLs

In [ ]:
# All functions to retrieve the informations

def get_title(soup):
    try:
        title = soup.find(id="bookTitle").contents[0].strip()
        return title
    except AttributeError:
        return ""

def get_series(soup):
    try:
        serie = soup.find(id="bookSeries").get_text().strip().replace("(","").replace(")","")
        regex = "#.*"
        serie = re.sub(regex, "",serie)
        return serie
    except AttributeError:
        return ""

def get_author(soup):
    try:
        author = soup.find_all(class_="authorName__container")
        author_list = " ".join([x.get_text().strip() for x in author])
        return author_list
    except AttributeError:
        return ""

def get_pages(soup):
    try:
        pages = soup.find(itemprop ="numberOfPages").get_text().split()[0]
        return int(pages)
    except AttributeError:
        return ""

def get_ratingValue(soup):
    try:
        ratingValue = soup.find(itemprop ="ratingValue").get_text().split()[0]
        return float(ratingValue)
    except AttributeError:
        return ""

def get_ratingCount(soup):
    try:
        test = []
        ratingCount = soup.find_all(class_ ="gr-hyperlink", href="#other_reviews")
        for x in ratingCount:
            test.append(x.get_text().split())
        return int(test[0][0].replace(",",""))
    except AttributeError:
        return ""

def get_reviewCount(soup):
    try:
        test = []
        ratingCount = soup.find_all(class_ ="gr-hyperlink", href="#other_reviews")
        for x in ratingCount:
            test.append(x.get_text().split())
        return int(test[1][0].replace(",",""))
    except AttributeError:
        return ""

def get_date(soup):
    try:
        regex = "by.*"
        date = soup.find_all(class_ = "row")
        date = re.sub(regex,"" ,str(date[1].contents[0]).replace("Published","")).strip()
        return date
    except AttributeError:
        return ""

def get_setting(soup):
    try:
        string = re.compile("/places/.")
        test = soup.find_all(href=string)
        places_list = " ".join([(str(a.contents[0])) for a in test])
        return places_list
    except AttributeError:
        return ""

def get_plot(soup):
    try:
        plot = soup.find(id="description")
        plot = plot.find_all(id= re.compile("freeText\.*"))

        final_plot = [] # It seems i need a list because joining on an empty string gives me a !"£$%&"@ blank

    # now there should not be a case where there are 3 span tags ...hopefully
        if len(plot) > 1:
            final_plot.append(plot[1].get_text())
        else:
            final_plot.append(plot[0].get_text())

        return final_plot[0]
    except AttributeError:
        return ""


def get_char(soup):
    try:
        string = re.compile("/characters/.")
        test = soup.find_all(href=string)

        char_list= " ".join([(str(a.contents[0])) for a in test])

        return char_list
    except AttributeError:
        return ""

def get_url(soup):
    try:
        regex = "https://www.goodreads.com/book/show/."
        ratingCount = soup.find(href =re.compile(regex), rel="canonical")
        return ratingCount.get('href')
    except AttributeError:
        return ""


## TSVs

In [ ]:
eng_count = 0
nb_html_articles = 100
page = 1
article_nb = 1

for k in range(1,100):

    path = "C:/Users/Guillaume/Desktop/ADM_HM3/page_"+str(page)
    os.chdir(path)

    for i in range(1,nb_html_articles+1):

        path = "C:/Users/Guillaume/Desktop/ADM_HM3/page_"+str(page)
        os.chdir(path)
        html = open("article_"+str(article_nb)+".html", "r", encoding="utf-8")
        soup = BeautifulSoup(html.read(), 'html.parser')

        article_nb += 1

        try:
            if detect(get_plot(soup)) == "en":
                eng_count += 1

                path = "C:/Users/Guillaume/Desktop/ADM_HM3/TSVs"
                os.chdir(path)
                  
                with open("article_"+str(eng_count)+".tsv", 'w', encoding="utf-8") as f_output:
                    test_list_tsv = [get_title(soup), 
                    get_series(soup), 
                    get_author(soup),
                    get_ratingValue(soup),
                    get_ratingCount(soup), 
                    get_reviewCount(soup),
                    get_plot(soup),
                    get_pages(soup),
                    get_date(soup),
                    get_setting(soup),
                    get_char(soup),
                    get_url(soup)]

                    header_list = ["bookTitle","bookSeries","bookAuthors","ratingValue","ratingCount","reviewCount","Plot","NumberofPages","Date","Characters","Setting","URL"]

                    tsv_output = csv.writer(f_output, delimiter='\t')
                    tsv_output.writerow(header_list)
                    tsv_output.writerow(test_list_tsv)
        except:
            language ="error"
        #ErrorCode.CantDetectError
    page = page + 1

In [ ]:
path = "C:/Users/Guillaume/Desktop/ADM_HM3/
os.chdir(path)

df = pd.DataFrame(columns = ["bookTitle","Plot","URL","Characters","ratingValue","Date"])

nb_articles = 26885

for i in range(1,nb_articles+1):
    article = open("article_"+str(i)+".tsv", "r", encoding="utf-8")
    data = pd.read_csv(article, sep = "\t", encoding = "utf-8")

    title = data["bookTitle"].iloc[0]
    plot = data["Plot"].iloc[0]
    url = data["URL"].iloc[0]
    characters = data["Setting"].iloc[0]
    ratingValue = data["ratingValue"].iloc[0]
    date = data["Date"].iloc[0]

    to_append = [title, plot, url,characters,ratingValue,date]
    df_length = len(df)
    df.loc[df_length] = to_append

# change to CVS file name you wish
df.to_csv("output.csv", index=False)